In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import (
    register_data_set,
)  # , register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import (
    RedshiftPointCasROIHeads,
    RedshiftPointROIHeads,
    RedshiftPDFROIHeads,
    return_lazy_model,
)
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser
from deepdisc.inference.predictors import return_predictor_transformer

In [2]:
# from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle
from rail.core.stage import RailStage



In [3]:
#cfgfile = "./tests/configs/solo/solo_res50_DC2.py"
cfgfile = "./configs/solo/solo_swin_DC2_weighted.py"

output_dir = "./"
output_name = "test"

trainfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5"
testfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test.hdf5"
metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/train_metadata.hdf5"
test_metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata.hdf5"


In [4]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [5]:
training = DS.add_data(
    "training", data=None, handle_class=TableHandle, path=trainfile
)  # ()["images"]
testing = DS.add_data("testing", data=None, handle_class=TableHandle, path=testfile)

#metadatahandle = DS.add_data("metadata", metadata, JsonHandle, path=metadatafile)
metadatahandle = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile)

In [6]:
deep_dict = dict(
    chunk_size=5,
    epoch=20,
    batch_size=1,
    output_dir="./",
    cfgfile = cfgfile,
    num_gpus=1,
    print_frequency=10,
    head_epochs=3,
    full_epochs=3,
    mile1=0,
    mile2=0,
    training_percent=1.0
)

In [7]:
Inform = DeepDiscInformer.make_stage(
    name="Inform_DeepDISC", model="test_informer.pkl", **deep_dict
)

In [ ]:
# Inform.inform(training, metadatahandle)
Inform.inform(training, metadatahandle) 

Caching data
Training head layers
[01/11 15:51:58 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/11 15:51:58 d2.data.common]: Serializing 800 elements to byte tensors and concatenating them all ...
[01/11 15:52:02 d2.data.common]: Serialized dataset takes 223.74 MiB
[01/11 15:52:02 d2.data.build]: Making batched data loader with batch_size=1
[01/11 15:52:02 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/11 15:52:02 d2.data.common]: Serializing 200 elements to byte tensors and concatenating them all ...
[01/11 15:52:09 d2.data.common]: Serialized dataset takes 55.56 MiB
[01/11 15:52:32 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/shared/hsc/detectron2/projects/ViTDet/model_final_246a82.pkl ...


Skip loading parameter 'backbone.bottom_up.patch_embed.proj.weight' to the model due to incompatible shapes: (128, 3, 4, 4) in the checkpoint but (128, 6, 4, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint 

[01/11 15:52:33 d2.engine.train_loop]: Starting training from iteration 0
Iteration:  10  data time:  0.263813559897244  loss time:  0.22456208523362875 dict_keys(['loss_cls_stage0', 'loss_box_reg_stage0', 'loss_cls_stage1', 'loss_box_reg_stage1', 'loss_cls_stage2', 'loss_box_reg_stage2', 'loss_mask', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.6415848135948181, 0.23508834838867188, 0.75870281457901, 0.48844820261001587, 0.6668554544448853, 1.1540870666503906, 0.5557609796524048, 0.6763800462640401, 0.5458568930625916, 0.22760748863220215] val loss:  0 lr:  [0.001]
Iteration:  20  data time:  0.36718152835965157  loss time:  0.2874174118041992 dict_keys(['loss_cls_stage0', 'loss_box_reg_stage0', 'loss_cls_stage1', 'loss_box_reg_stage1', 'loss_cls_stage2', 'loss_box_reg_stage2', 'loss_mask', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.5144137740135193, 0.2540614902973175, 0.4824583828449249, 0.6232739686965942, 0.43338853120803833, 1.468063473701477, 0.4116071462631225

### Inference


In [ ]:
# metadatahandle = DS.add_data(
#     "metadata", test_metadata, JsonHandle, path=test_metadatafile
# )

In [ ]:
# Estimator = DeepDiscPDFEstimator.make_stage(
#     name="DeepDiscEstimator",
#     model=Inform.get_handle("model"),
#     # hdf5_groupname="images",
#     **deep_dict,
# )

In [ ]:
# Estimator.config

In [ ]:
# results = Estimator.estimate(testing, metadatahandle)

In [ ]:
# res = results.read()

In [ ]:
# truth = Estimator.get_handle("truth")

In [ ]:
# ztrue = truth.data["redshift"]

In [7]:
#from rail.core.data import ModelHandle
MH = ModelHandle('model',path='./test_informer.pkl')
data = MH.read()
#cfgfile = "./configs/solo/solo_swin_DC2.py"
#cfg = LazyConfig.load(cfgfile)
#checkpoint = MH()['nnmodel']

In [8]:
#data

## Test new chunking algorithm

In [7]:
deep_estimation_dict = dict(
    chunk_size=100,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
    include_ids=True
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    #model=Inform.get_handle("model"),
    model='/home/g4merz/DC2/model_tests/Swin_NG3_weighted_informer.pkl',
    **deep_estimation_dict,
)

In [8]:
#test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test.hdf5"

test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
#metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata.hdf5"

metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

In [11]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

Inserting handle into data store.  model: ./test_informer.pkl, DeepDiscEstimatorWithChunks
Caching data
Processing chunk (start:end) - (0:5)
Matching objects
Adding PDFs to ensemble
Adding true Z to ensemble
Adding object IDs to ensemble
Writing out this temporary ensemble to disk
Processing chunk (start:end) - (5:10)
Matching objects
Adding PDFs to ensemble
Adding true Z to ensemble
Adding object IDs to ensemble
Writing out this temporary ensemble to disk
Processing chunk (start:end) - (10:15)
Matching objects
Adding PDFs to ensemble
Adding true Z to ensemble
Adding object IDs to ensemble
Writing out this temporary ensemble to disk
Processing chunk (start:end) - (15:20)
Matching objects
Adding PDFs to ensemble
Adding true Z to ensemble
Adding object IDs to ensemble
Writing out this temporary ensemble to disk
Processing chunk (start:end) - (20:25)
Matching objects
Adding PDFs to ensemble
Adding true Z to ensemble
Adding object IDs to ensemble
Writing out this temporary ensemble to disk

In [12]:
res_ens = results_from_chunks.read()
res_ens.npdf

87

In [13]:
res_ens.ancil

{'mode': array([[0.52763819],
        [0.57788945],
        [0.55276382],
        [0.60301508],
        [0.65326633],
        [0.65326633],
        [0.50251256],
        [0.6281407 ],
        [0.6281407 ],
        [0.47738693],
        [0.65326633],
        [0.65326633],
        [0.55276382],
        [0.55276382],
        [0.60301508],
        [0.70351759],
        [0.85427136],
        [0.65326633],
        [0.6281407 ],
        [0.65326633],
        [0.55276382],
        [0.52763819],
        [0.60301508],
        [0.70351759],
        [0.75376884],
        [0.55276382],
        [0.52763819],
        [0.60301508],
        [0.52763819],
        [0.57788945],
        [0.6281407 ],
        [0.6281407 ],
        [0.60301508],
        [0.47738693],
        [0.65326633],
        [0.67839196],
        [0.67839196],
        [0.65326633],
        [0.50251256],
        [0.60301508],
        [0.60301508],
        [0.57788945],
        [0.57788945],
        [0.52763819],
        [0.60301508],
  

In [ ]:
output = EstimatorWithChunks.get_handle("output")

In [ ]:
output.data.ancil

In [ ]:
dtest = np.arange(9)
output.data.add_to_ancil(dict(ids=dtest))
output.data.ancil

In [ ]:
#help(output.data)

## Evaluation

In [ ]:
from rail.evaluation.evaluator import Evaluator

In [ ]:
deep_eval_dict = dict(
    chunk_size=100,
    zmin=-1,
    zmax=5,
    nzbins=200,
    epochs=20,
    output_name="test_evaluator",
    point_metrics="",
)

In [ ]:
DeepEvaluator = Evaluator.make_stage(name="DeepDiscEvaluator", **deep_eval_dict)

In [ ]:
eval_res = DeepEvaluator.evaluate(res, truth)

In [ ]:
eval_res.data

In [ ]:
from qp.metrics.pit import PIT
from utils import *  # plot_pit_qq, ks_plot

pitobj = PIT(res, ztrue)
pit_out_rate = pitobj.evaluate_PIT_outlier_rate()

In [ ]:
from qp.ensemble import Ensemble


class Sample(Ensemble):
    """Expand qp.Ensemble to append true redshifts
    array, metadata, and specific plots."""

    def __init__(
        self, pdfs, zgrid, ztrue, photoz_mode=None, code="", name="", n_quant=100
    ):
        """Class constructor

        Parameters
        ----------
        pdfs: `ndarray`
            photo-z PDFs array, shape=(Ngals, Nbins)
        zgrid: `ndarray`
            PDF bins centers, shape=(Nbins,)
        ztrue: `ndarray`
            true redshifts, shape=(Ngals,)
        photoz_mode: `ndarray`
            photo-z (PDF mode), shape=(Ngals,)
        code: `str`, (optional)
            algorithm name (for plot legends)
        name: `str`, (optional)
            sample name (for plot legends)
        """

        super().__init__(interp, data=dict(xvals=zgrid, yvals=pdfs))
        self._pdfs = pdfs
        self._zgrid = zgrid
        self._ztrue = ztrue
        self._photoz_mode = photoz_mode
        self._code = code
        self._name = name
        self._n_quant = n_quant
        self._pit = None
        self._qq = None

    @property
    def code(self):
        """Photo-z code/algorithm name"""
        return self._code

    @property
    def name(self):
        """Sample name"""
        return self._name

    @property
    def ztrue(self):
        """True redshifts array"""
        return self._ztrue

    @property
    def zgrid(self):
        """Redshift grid (binning)"""
        return self._zgrid

    @property
    def photoz_mode(self):
        """Photo-z (mode) array"""
        return self._photoz_mode

    @property
    def n_quant(self):
        return self._n_quant

    @property
    def pit(self):
        if self._pit is None:
            pit_array = np.array(
                [self[i].cdf(self.ztrue[i])[0][0] for i in range(len(self))]
            )
            self._pit = pit_array
        return self._pit

    @property
    def qq(self, n_quant=100):
        q_theory = np.linspace(0.0, 1.0, n_quant)
        q_data = np.quantile(self.pit, q_theory)
        self._qq = (q_theory, q_data)
        return self._qq

    def __len__(self):
        if len(self._ztrue) != len(self._pdfs):
            raise ValueError("Number of pdfs and true redshifts do not match!!!")
        return len(self._ztrue)

    def __str__(self):
        code_str = f"Algorithm: {self._code}"
        name_str = f"Sample: {self._name}"
        line_str = "-" * (max(len(code_str), len(name_str)))
        text = str(
            line_str
            + "\n"
            + name_str
            + "\n"
            + code_str
            + "\n"
            + line_str
            + "\n"
            + f"{len(self)} PDFs with {len(self.zgrid)} probabilities each \n"
            + f"qp representation: {self.gen_class.name} \n"
            + f"z grid: {len(self.zgrid)} z values from {np.min(self.zgrid)} to {np.max(self.zgrid)} inclusive"
        )
        return text

    def plot_pdfs(self, gals, show_ztrue=True, show_photoz_mode=False):
        colors = plot_pdfs(
            self, gals, show_ztrue=show_ztrue, show_photoz_mode=show_photoz_mode
        )
        return colors

    def plot_old_valid(self, gals=None, colors=None):
        old_metrics_table = plot_old_valid(self, gals=gals, colors=colors)
        return old_metrics_table

    def plot_pit_qq(
        self,
        bins=None,
        label=None,
        title=None,
        show_pit=True,
        show_qq=True,
        show_pit_out_rate=True,
        savefig=False,
    ):
        """Make plot PIT-QQ as Figure 2 from Schmidt et al. 2020."""
        fig_filename = plot_pit_qq(
            self,
            bins=bins,
            label=label,
            title=title,
            show_pit=show_pit,
            show_qq=show_qq,
            show_pit_out_rate=show_pit_out_rate,
            savefig=savefig,
        )
        return fig_filename

In [ ]:
def plot_pit_qq(
    pdfs,
    zgrid,
    ztrue,
    bins=None,
    title=None,
    code=None,
    show_pit=True,
    show_qq=True,
    pit_out_rate=None,
    savefig=False,
) -> str:
    """Quantile-quantile plot
        Ancillary function to be used by class Metrics.
    ​
        Parameters
        ----------
        pit: `PIT` object
            class from metrics.py
        bins: `int`, optional
            number of PIT bins
            if None, use the same number of quantiles (sample.n_quant)
        title: `str`, optional
            if None, use formatted sample's name (sample.name)
        label: `str`, optional
            if None, use formatted code's name (sample.code)
        show_pit: `bool`, optional
            include PIT histogram (default=True)
        show_qq: `bool`, optional
            include QQ plot (default=True)
        pit_out_rate: `ndarray`, optional
            print metric value on the plot panel (default=None)
        savefig: `bool`, optional
            save plot in .png file (default=False)
    """

    if bins is None:
        bins = 100
    if title is None:
        title = ""

    if code is None:
        code = ""
        label = ""
    else:
        label = code + "\n"

    if pit_out_rate is not None:
        try:
            label += "PIT$_{out}$: "
            label += f"{float(pit_out_rate):.4f}"
        except:
            print("Unsupported format for pit_out_rate.")

    plt.figure(figsize=[4, 5])
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
    ax0 = plt.subplot(gs[0])
    sample = Sample(pdfs, zgrid, ztrue)

    if show_qq:
        ax0.plot(
            sample.qq[0], sample.qq[1], c="r", linestyle="-", linewidth=3, label=label
        )
        ax0.plot([0, 1], [0, 1], color="k", linestyle="--", linewidth=2)
        ax0.set_ylabel("Q$_{data}$", fontsize=18)
        plt.ylim(-0.001, 1.001)
    plt.xlim(-0.001, 1.001)
    plt.title(title)
    if show_pit:
        fzdata = Ensemble(interp, data=dict(xvals=zgrid, yvals=pdfs))
        pitobj = PIT(fzdata, ztrue)
        pit_vals = np.array(pitobj.pit_samps)
        pit_out_rate = pitobj.evaluate_PIT_outlier_rate()

        try:
            y_uni = float(len(pit_vals)) / float(bins)
        except:
            y_uni = float(len(pit_vals)) / float(len(bins))
        if not show_qq:
            ax0.hist(pit_vals, bins=bins, alpha=0.7, label=label)
            ax0.set_ylabel("Number")
            ax0.hlines(y_uni, xmin=0, xmax=1, color="k")
            plt.ylim(
                0,
            )  # -0.001, 1.001)
        else:
            ax1 = ax0.twinx()
            ax1.hist(pit_vals, bins=bins, alpha=0.7)
            ax1.set_ylabel("Number")
            ax1.hlines(y_uni, xmin=0, xmax=1, color="k")
    leg = ax0.legend(handlelength=0, handletextpad=0, fancybox=True)
    for item in leg.legendHandles:
        item.set_visible(False)
    if show_qq:
        ax2 = plt.subplot(gs[1])
        ax2.plot(
            sample.qq[0],
            (sample.qq[1] - sample.qq[0]),
            c="r",
            linestyle="-",
            linewidth=3,
        )
        plt.ylabel("$\Delta$Q", fontsize=18)
        ax2.plot([0, 1], [0, 0], color="k", linestyle="--", linewidth=2)
        plt.xlim(-0.001, 1.001)
        plt.ylim(
            np.min([-0.12, np.min(sample.qq[1] - sample.qq[0]) * 1.05]),
            np.max([0.12, np.max(sample.qq[1] - sample.qq[0]) * 1.05]),
        )
    if show_pit:
        if show_qq:
            plt.xlabel("Q$_{theory}$ / PIT Value", fontsize=18)
        else:
            plt.xlabel("PIT Value", fontsize=18)
    else:
        if show_qq:
            plt.xlabel("Q$_{theory}$", fontsize=18)
    if savefig:
        fig_filename = str("plot_pit_qq_" + f"{(code).replace(' ', '_')}.png")
        plt.savefig(fig_filename)
    else:
        fig_filename = None

    return fig_filename

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
from qp import interp


zgrid = np.linspace(-1, 5, 200)
pdfs = res.objdata()["yvals"]
plot_pit_qq(
    pdfs,
    zgrid,
    ztrue,
    title="PIT-QQ - toy data",
    code="DeepDISC",
    pit_out_rate=pit_out_rate,
    savefig=False,
)

In [ ]:
res.objdata()